# 1) intro 생성 테스트

In [1]:
from Gen_intro import llm_moderator_generate_intro

# 가짜 입력값 JSON 데이터
sample_contents = {
    "mode_configuration": {
        "dialogue_style": "Debate_Mode",
        "interaction_goal": "Concept_Verification",
        "goal_description": "교과서적 개념에 대해 사용자가 설명하고, 상대 에이전트가 이를 반박하거나 허점을 지적하며 개념의 정확성을 검증함"
    },
    "content_context": {
        "target_topic": {
            "keyword": "DQN (Deep Q-Network)",
            "description": "DQN의 주요 메커니즘인 Experience Replay와 Target Network의 필요성에 대한 개념적 방어"
        },
        "knowledge_boundary": "Lecture_Only"
    },
    "session_rules": {
        "max_turns": 5,
        "difficulty_parameter": {
            "level": "High",
            "custom_constraints": [
                "논리적 비약이 있을 경우 즉시 지적할 것",
                "수식보다는 개념의 인과관계 위주로 반박할 것"
            ]
        }
    }
}

# 함수 입력 파라미터로 변수 정의 (가짜 입력값)
topic_keyword = sample_contents["content_context"]["target_topic"]["keyword"]
topic_description = sample_contents["content_context"]["target_topic"]["description"]
dialogue_style = sample_contents["mode_configuration"]["dialogue_style"]
rules = sample_contents["session_rules"]

response = llm_moderator_generate_intro(topic_keyword, rules, topic_description, dialogue_style)


ClientError: 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Your API key was reported as leaked. Please use another API key.', 'status': 'PERMISSION_DENIED'}}

In [4]:
print(response)

{
"message": "지금부터 DQN(Deep Q-Network)의 핵심 기법인 Experience Replay와 Target Network의 필요성에 대한 심화 비판 토론을 시작합니다. 본 세션은 난이도 '상'으로 설정되었으며 총 5턴 동안 진행됩니다. 이번 토론에서는 단순한 수식의 나열보다는 각 개념 사이의 인과관계를 중심으로 논리적 비약이 없는지 엄격하게 검증할 것입니다. 먼저, DQN에서 이 두 메커니즘이 제거되었을 때 발생하는 학습 불안정성의 근본적인 원인이 무엇인지 논리적으로 증명하며 입론을 시작해 주십시오.",
"opening_strategy": "Academic defense framing for a high-difficulty session. It explicitly warns the user that logical causality will be prioritized over formulas, establishing a rigorous and critical atmosphere."
}


# 2) check Safety 생성 테스트

In [16]:
from Check_safety import llm_moderator_check_safety

user_input = "안녕?"
topic_keyword = "DQN (Deep Q-Network)"
topic_description = "DQN의 주요 메커니즘인 Experience Replay와 Target Network의 필요성에 대한 개념적 방어"

safety_check_str = llm_moderator_check_safety(user_input, topic_keyword, topic_description)

In [17]:
print(safety_check_str)

{"is_valid":false,"reason":"인삿말이나 잡담은 토론과 무관합니다. DQN의 Experience Replay와 Target Network의 필요성에 대한 구체적인 의견이나 논거를 제시해 주세요.","violation_type":"Off_Topic"}


# 3) Eval_assess_step 테스트

In [22]:
from Eval_assess_step import llm_evaluator_assess_step
#user_input = "DQN은 Experience Replay를 사용하여 샘플 간의 상관관계를 줄이고, Target Network를 통해 학습의 안정성을 확보합니다. Experience Replay는 과거 경험을 버퍼에 저장했다가 무작위로 샘플링하여 학습하므로 연속된 샘플 간의 상관성을 제거할 수 있습니다."
user_input = "몰라 어케알아 그딴걸"
last_attack = "DQN에서 Experience Replay가 정말 필요한가요? 그냥 최신 데이터만 사용하면 더 효율적이지 않을까요?"

lecture_material = "/Users/jhkim/Desktop/Debate_Agent/phase2/lecture_materials/DQN_lecture.txt"

dialogue_style = "Concept_Learning"

difficulty = "Medium"

eval_result_str = llm_evaluator_assess_step(user_input, last_attack, lecture_material, dialogue_style, difficulty)

In [23]:
print(eval_result_str)

{"score_delta":-8,"rationale":"DQN의 핵심 개념인 Experience Replay의 필요성(샘플 간 상관관계 제거 및 안정적 학습)에 대해 전혀 설명하지 않고 답변을 회피했습니다. 중급 난이도에서 요구되는 최소한의 개념 설명이 이루어지지 않았습니다.","detected_logic_flaw":"Refusal to Answer"}


# 4) Gen_attack 테스트

In [24]:
from Debater_Generate_Attack import llm_debater_generate_attack

# 가짜 데이터 (테스트용)
user_input = "Experience Replay는 과거 경험을 재사용하지만, 타겟 네트워크를 주기적으로 업데이트함으로써 stale policy 문제를 완화하고, 샘플 간 상관관계를 제거하여 학습 안정성을 높입니다."

conversation_history = [
    # Turn 1 - AI의 첫 공격
    {
        "role": "agent",
        "content": "DQN의 Experience Replay가 샘플 효율성을 높인다고 주장하셨는데, 과거 경험을 재사용하는 것이 오히려 오래된 정책(stale policy)으로 학습하게 만들어 수렴을 방해하는 것 아닙니까?",
        "meta": {"tactic": "Logic_Error_Point"}
    },
]

lecture_material = "/Users/jhkim/Desktop/Debate_Agent/phase2/lecture_materials/DQN_lecture.txt"

dialogue_style = "Concept_Learning"

difficulty = "High"

response = llm_debater_generate_attack(user_input, conversation_history, lecture_material, difficulty, dialogue_style)

In [25]:
print(response)

{
  "argument": "타겟 네트워크가 Experience Replay의 stale policy 문제를 완화한다고요? 오히려 타겟 네트워크는 학습의 안정성을 위해 의도적으로 'stale'한 파라미터를 고정하여 사용하는 기법입니다. 리플레이 버퍼에 쌓인 과거의 데이터가 현재 정책과 동떨어져 발생하는 본질적인 편향(bias) 문제를, 단순히 타겟 값을 고정하는 타겟 네트워크가 어떻게 직접적으로 '완화'할 수 있다는 건지 논리적 근거를 제시하십시오. 두 기법은 서로 다른 문제를 해결하기 위한 장치인데, 개념을 교묘하게 섞어서 얼버무리는 것 아닙니까?",
  "attack_tactic": "Logic_Error_Point"
}


# 5) Eval_assess_closing 테스트

In [26]:
from Eval_assess_closing import llm_evaluator_assess_closing

final_user_speech = "모르겠다, 니 다해라"
history = [
    {
        "role": "agent",
        "content": "DQN의 Experience Replay가 샘플 효율성을 높인다고 주장하셨는데, 과거 경험을 재사용하는 것이 오히려 오래된 정책(stale policy)으로 학습하게 만들어 수렴을 방해하는 것 아닙니까?",
        "meta": {"tactic": "Logic_Error_Point"}
    },
    
    {
        "role": "user",
        "content": "Experience Replay는 i.i.d 가정을 만족시키기 위해 필요합니다. 연속된 샘플 간의 correlation을 깨뜨려 학습의 안정성을 높입니다."
    },
    
    {
        "role": "agent",
        "content": "i.i.d를 만족시킨다는 주장은 이해하겠습니다만, 그렇다면 왜 굳이 Target Network까지 필요한 겁니까? Experience Replay만으로도 충분하지 않나요? 두 메커니즘의 역할이 중복되는 것 아닙니까?",
        "meta": {"tactic": "Deep_Inquiry"}
    },
    
    {
        "role": "user",
        "content": "Target Network는 다른 목적입니다. Q-learning의 업데이트 과정에서 target 값이 계속 변하면 moving target 문제가 발생합니다. Target Network를 고정시켜서..."
    },
    
    {
        "role": "agent",
        "content": "그 설명은 피상적입니다. Moving target이 문제라면 learning rate를 낮추는 것으로도 해결 가능하지 않습니까? Target Network의 고유한 필요성을 수식적으로 증명하지 못하고 계십니다.",
        "meta": {"tactic": "Sarcastic_Refutation"}
    }
]
lecture_material = "/Users/jhkim/Desktop/Debate_Agent/phase2/lecture_materials/DQN_lecture.txt"
dialogue_style = "Concept_Learning"
difficulty = "High"

response = llm_evaluator_assess_closing(final_user_speech, history, lecture_material, dialogue_style, difficulty)

In [27]:
print(response)

{"score_delta": -10, "final_impression": "학습 중인 핵심 개념에 대한 논리적 방어를 포기하고 대화를 중단하였으므로, 비판적 탐구와 개념 이해의 요건을 모두 충족하지 못했습니다."}


# 6) Generate_Summary 테스트

In [3]:
from Generate_Summary import llm_moderator_generate_summary

# contents 생성
history = [
    {
        "role": "agent",
        "content": "DQN에서 Experience Replay를 사용하면 샘플 효율성이 높아진다고 하셨는데, 오래된 경험을 재사용하는 것이 오히려 현재 정책과 맞지 않아 학습을 방해하는 것 아닙니까?"
    },
    {
        "role": "user",
        "content": "Experience Replay는 i.i.d 가정을 만족시키기 위해 필요합니다. 연속된 샘플 간의 correlation을 제거합니다."
    },
    {
        "role": "agent",
        "content": "그렇다면 Target Network는 왜 필요합니까? Replay Buffer만으로 충분하지 않나요?"
    },
    {
        "role": "user",
        "content": "Target Network는 moving target 문제를 해결합니다. Q-learning에서 target 값이 계속 변하면 학습이 불안정해집니다."
    },
    {
        "role": "agent",
        "content": "Moving target 문제라고요? Learning rate를 낮추면 되지 않습니까?"
    },
    {
        "role": "user",
        "content": "Learning rate만으로는 부족합니다. Bellman update에서 target이 고정되어야 수렴이 보장됩니다."
    },
    {
        "role": "agent",
        "content": "Overestimation bias는 어떻게 해결하셨나요?"
    },
    {
        "role": "user",
        "content": "그건... DQN의 한계입니다. Double DQN에서 해결되었습니다."
    },
    {
        "role": "user",
        "content": "[최후 변론] DQN은 Experience Replay와 Target Network를 통해 deep learning을 강화학습에 안정적으로 결합한 혁명적인 방법입니다."
    }
]
evaluation_logs = [
    {
        "turn": 1,
        "score_now": 55,
        "reason": "i.i.d 가정과 correlation 제거를 정확히 언급함."
    },
    {
        "turn": 2,
        "score_now": 63,
        "reason": "Target Network의 역할을 명확히 구분하고 moving target 문제를 지적함."
    },
    {
        "turn": 3,
        "score_now": 66,
        "reason": "Bellman equation을 언급했으나 수식적 증명까지는 도달하지 못함."
    },
    {
        "turn": 4,
        "score_now": 64,
        "reason": "Overestimation bias를 제대로 방어하지 못하고 한계라고만 인정함."
    },
    {
        "turn": "Final",
        "score_now": 70,
        "reason": "DQN의 역사적 의의를 강조한 설득력 있는 최후 변론."
    }
]
final_status = "WIN"

response = llm_moderator_generate_summary(history, evaluation_logs, final_status)


In [4]:
print(response)

{
  "summary_text": "이번 토론은 DQN의 핵심 아키텍처를 주제로 진행되었으며, 사용자님은 기술적 개념에 대한 정확한 이해를 바탕으로 승리(WIN)를 거두셨습니다. 특히 Experience Replay의 필요성을 'i.i.d 가정 만족'과 '샘플 간 상관관계(correlation) 제거'로 명확히 정의하며 논리적 우위를 점하신 점이 훌륭했습니다. 에이전트가 제기한 Target Network의 불필요성 논란에 대해서도 'moving target 문제'와 'Bellman update의 수렴성'을 근거로 논리적으로 방어해 내셨습니다. 비록 'Overestimation bias' 질문에서 구체적인 수식적 원인 분석보다는 'Double DQN'이라는 대안 제시와 한계 인정에 그쳤으나, 전체적인 흐름을 유지하는 데는 충분했습니다. 마지막 변론에서 DQN이 딥러닝과 강화학습을 결합한 '혁명적인 방법'임을 강조하며 토론의 가치를 높인 점이 승리의 결정적 요인이었습니다.",
  "key_takeaways": [
    "i.i.d (Independent and Identically Distributed)",
    "Moving Target Problem",
    "Overestimation Bias"
  ]
}
